# Import

In [1]:
import pandas as pd
import gtfs_kit as gk
import helper #homemade function file helper.py
import warnings
import numpy as np
from tqdm import tqdm

warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

# Main Dataset Creation

In [2]:
# Load GTFS Static
feed = gk.feed.read_feed('../processed_files/preprocessing_gtfs_static.zip',dist_units="km")
routes = gk.routes.get_routes(feed)
stop_times = feed.get_stop_times()
trips = feed.get_trips()
stops = feed.get_stops()
shapes = feed.shapes
calendar_dates = feed.calendar_dates
calendar = feed.calendar

# Load Delays
delays = pd.read_csv('../processed_files/delays_preprocessing_expanded.csv',index_col=[0])
# Load Weather
weather = pd.read_csv('../processed_files/measurments_preprocessing.csv',index_col=[0])
#weather = pd.read_csv('prova.csv',index_col=[0])

In [3]:
main_dataset = stop_times.copy()
main_dataset = pd.merge(main_dataset, trips, on=['trip_id','trip_id']).reset_index(drop=True)
main_dataset = pd.merge(main_dataset, calendar_dates, on=['service_id','service_id']).reset_index(drop=True)
main_dataset = pd.merge(main_dataset, stops, on=['stop_id','stop_id']).reset_index(drop=True)
main_dataset = main_dataset.drop(['route_id', 'service_id','trip_headsign', 'shape_id',
                                 'geometry','stop_name'], axis=1)

### Keep only rows where date are compatible with delays and weather
main_dataset.shape

(1666671, 15)

In [4]:
main_dataset.sort_values('trip_short_name',ascending=False)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,shape_dist_traveled,time_diff,speed,dist_diff,trip_short_name,direction_id,date,stop_lat,stop_lon,stop_code
1115697,4118_296557,08:39:00,08:40:00,S06300_1,5,12.46049,0.083333,58.707600,4.89230,19601,1,20231028,44.116437,10.409185,S06300
129612,4118_296557,09:34:00,09:34:00,S06404_1,14,57.39790,0.100000,52.339300,5.23393,19601,1,20231028,43.837397,10.506153,S06404
728926,4118_296557,09:16:00,09:17:00,S06310_1,11,40.97815,0.050000,70.131000,3.50655,19601,1,20231028,43.959490,10.507770,S06310
844357,4118_296557,09:07:00,09:08:00,S06308_1,9,34.09033,0.116667,40.338514,4.70616,19601,1,20231028,44.003472,10.551756,S06308
611779,4118_296557,09:27:00,09:28:00,S06313_1,13,52.16397,0.050000,77.064200,3.85321,19601,1,20231028,43.875088,10.531057,S06313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36247,2218_224183,08:40:00,08:40:00,S06404_1,5,23.13283,0.100000,80.185300,8.01853,06123,0,20181031,43.837397,10.506153,S06404
367409,2218_224183,08:20:00,08:21:00,S06501_1,2,2.22396,0.083333,26.687520,2.22396,06123,0,20181031,43.723446,10.387754,S06501
59231,2455_252637,08:40:00,08:40:00,S06404_1,5,23.13283,0.100000,80.185300,8.01853,06123,0,20191102,43.837397,10.506153,S06404
367411,2218_224183,08:20:00,08:21:00,S06501_1,2,2.22396,0.083333,26.687520,2.22396,06123,0,20181103,43.723446,10.387754,S06501


In [5]:
main_dataset = main_dataset.rename(columns={'trip_short_name': 'codice','stop_id':'codice_stazione','date':'data_giorno'})

In [6]:
main_dataset = main_dataset.loc[(main_dataset["data_giorno"] >= "20200110") & (main_dataset["data_giorno"] <= "20231031")]

In [7]:
main_dataset.shape

(891949, 15)

In [8]:
#Set Join columns to same dtype
delays["codice"] = delays["codice"].astype(str)
delays["data_giorno"] = delays["data_giorno"].astype(str)

#Join on Delays
main_dataset = pd.merge(main_dataset, delays, on=['codice', 'data_giorno','codice_stazione'])

In [9]:
main_dataset.shape

(322318, 19)

In [10]:
#Calcolo orario arrotondato per join con weather.
main_dataset['orario'] = main_dataset.loc[:, 'arrival_time']
max_index_for_display = main_dataset.index.max()
for index, row  in tqdm(main_dataset.iterrows()):
    orario_to_split = main_dataset.loc[index,"orario"].split(":")
    if orario_to_split[1] >= "30":
        orario_to_split[0]= str( int(orario_to_split[0]) + 1 )
    main_dataset.loc[index,"orario"] = orario_to_split[0] + ":00"

322318it [00:12, 25862.85it/s]


In [11]:
#Join on Weather
weather['data_giorno'] = weather['data_giorno'].astype(str)
main_dataset = pd.merge(main_dataset, weather, on=['data_giorno','orario','codice_stazione'])
main_dataset.shape



(291999, 36)

In [12]:
#Sort Dataset
main_dataset = main_dataset.sort_values(['trip_id','data_giorno','stop_sequence'],ascending=True)
main_dataset = main_dataset.reset_index(drop=True)

In [13]:
# We create an empty column for time different, speed, distance 
main_dataset = main_dataset.assign(temperature_linea=None)
main_dataset = main_dataset.assign(app_temp_linea=None)
main_dataset = main_dataset.assign(rain_linea=None)
main_dataset = main_dataset.assign(snow_fall_linea=None)
main_dataset = main_dataset.assign(snow_depth_linea=None)
main_dataset = main_dataset.assign(humidity_linea=None)
main_dataset = main_dataset.assign(precipitation_linea=None)
main_dataset = main_dataset.assign(wmo_code_linea=None)
main_dataset = main_dataset.assign(wind_speed_10m_linea=None)
main_dataset = main_dataset.assign(wind_speed_100m_linea=None)
main_dataset = main_dataset.assign(wind_dir_10m_linea=None)
main_dataset = main_dataset.assign(wind_dir_100m_linea=None)
main_dataset = main_dataset.assign(wind_gusts_linea=None)
main_dataset = main_dataset.assign(soil_temperature_linea=None)

max_index_for_display = main_dataset.index.max()

# Calculating Distance, Time and avg Speed between last stop and current stop
for index, row  in tqdm(main_dataset.iterrows()):
    if row['stop_sequence'] == 1:
        main_dataset['temperature_linea'][index] = row['temperature']
        main_dataset['app_temp_linea'][index] = row['app_temp']
        main_dataset['rain_linea'][index] = row['rain']
        main_dataset['snow_fall_linea'][index] = row['snow_fall']
        main_dataset['snow_depth_linea'][index] = row['snow_depth']
        main_dataset['humidity_linea'][index] = row['humidity']
        main_dataset['precipitation_linea'][index] = row['precipitation']
        main_dataset['wind_speed_10m_linea'][index] = row['wind_speed_10m']
        main_dataset['wind_speed_100m_linea'][index] = row['wind_speed_100m']
        main_dataset['wind_dir_10m_linea'][index] = row['wind_dir_10m']
        main_dataset['wind_dir_100m_linea'][index] = row['wind_dir_100m']
        main_dataset['wmo_code_linea'][index] = row['wmo_code']
        main_dataset['wind_gusts_linea'][index] = row['wind_gusts']
        main_dataset['soil_temperature_linea'][index] = row['soil_temperature']
    else:
        main_dataset['temperature_linea'][index] = ((row['temperature'] + main_dataset.loc[index - 1,'temperature']) / 2) 
        main_dataset['app_temp_linea'][index] = ((row['app_temp'] + main_dataset.loc[index - 1,'app_temp']) / 2) 
        main_dataset['rain_linea'][index] = ((row['rain'] + main_dataset.loc[index - 1,'rain']) / 2) 
        main_dataset['snow_fall_linea'][index] = ((row['snow_fall'] + main_dataset.loc[index - 1,'snow_fall']) / 2) 
        main_dataset['snow_depth_linea'][index] = ((row['snow_depth'] + main_dataset.loc[index - 1,'snow_depth']) / 2) 
        main_dataset['precipitation_linea'][index] = ((row['precipitation'] + main_dataset.loc[index - 1,'precipitation']) / 2) 
        main_dataset['humidity_linea'][index] = ((row['humidity'] + main_dataset.loc[index - 1,'humidity']) / 2)
        main_dataset['wind_speed_10m_linea'][index] = ((row['wind_speed_10m'] + main_dataset.loc[index - 1,'wind_speed_10m']) / 2) 
        main_dataset['wind_speed_100m_linea'][index] =((row['wind_speed_100m'] + main_dataset.loc[index - 1,'wind_speed_100m']) / 2)  
        main_dataset['wind_dir_10m_linea'][index] = ((row['wind_dir_10m'] + main_dataset.loc[index - 1,'wind_dir_10m']) / 2) 
        main_dataset['wind_dir_100m_linea'][index] = ((row['wind_dir_100m'] + main_dataset.loc[index - 1,'wind_dir_100m']) / 2)
        main_dataset['wmo_code_linea'][index] = ((row['wmo_code'] + main_dataset.loc[index - 1,'wmo_code']) / 2) 
        main_dataset['wind_gusts_linea'][index] = ((row['wind_gusts'] + main_dataset.loc[index - 1,'wind_gusts']) / 2) 
        main_dataset['soil_temperature_linea'][index] = ((row['soil_temperature'] + main_dataset.loc[index - 1,'soil_temperature']) / 2)

291999it [2:29:25, 32.57it/s]


In [14]:
main_dataset

,trip_id,arrival_time,departure_time,codice_stazione,stop_sequence,shape_dist_traveled,time_diff,speed,dist_diff,codice,...,snow_depth_linea,humidity_linea,precipitation_linea,wmo_code_linea,wind_speed_10m_linea,wind_speed_100m_linea,wind_dir_10m_linea,wind_dir_100m_linea,wind_gusts_linea,soil_temperature_linea
0,3014_278962,16:42:00,16:42:00,S06404_1,1,0.00000,0.000000,0.000000,0.00000,18586,...,0.0,93,0.5,53,8.3,14.6,72,81,15.1,11.1
1,3014_278962,16:48:00,16:49:00,S06402_1,2,8.01853,0.100000,80.185300,8.01853,18586,...,0.0,93.0,0.5,53.0,8.3,14.6,72.0,81.0,15.1,11.1
2,3014_278962,16:57:00,16:58:00,S06400_1,3,14.83468,0.133333,51.121125,6.81615,18586,...,0.0,94.5,0.3,52.0,6.9,13.5,69.5,77.0,15.45,10.8
3,3014_278962,17:08:00,17:09:00,S06501_1,4,20.90887,0.166667,36.445140,6.07419,18586,...,0.0,96.5,0.05,27.0,6.3,13.85,71.0,77.0,15.45,10.35
4,3014_278962,17:13:00,17:13:00,S06500_1,5,23.53195,0.066667,39.346200,2.62308,18586,...,0.0,97.0,0.0,3.0,7.1,15.3,75.0,81.0,15.1,10.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291994,4118_296638,17:42:00,17:42:00,S06404_1,1,0.00000,0.000000,0.000000,0.00000,18594,...,0.0,73,0.4,51,16.9,30.5,259,256,32.4,17.0
291995,4118_296638,17:48:00,17:49:00,S06402_1,2,8.01853,0.100000,80.185300,8.01853,18594,...,0.0,73.0,0.4,51.0,16.9,30.5,259.0,256.0,32.4,17.0
291996,4118_296638,17:57:00,17:58:00,S06400_1,3,14.83468,0.133333,51.121125,6.81615,18594,...,0.0,74.5,0.2,26.5,14.65,28.35,259.5,256.5,34.2,17.15
291997,4118_296638,18:07:00,18:08:00,S06501_1,4,20.90887,0.150000,40.494600,6.07419,18594,...,0.0,73.5,0.0,2.0,13.95,28.45,259.0,256.5,33.85,17.7


In [15]:
main_dataset=main_dataset.dropna().reset_index(drop=True)


In [16]:
main_dataset

,trip_id,arrival_time,departure_time,codice_stazione,stop_sequence,shape_dist_traveled,time_diff,speed,dist_diff,codice,...,snow_depth_linea,humidity_linea,precipitation_linea,wmo_code_linea,wind_speed_10m_linea,wind_speed_100m_linea,wind_dir_10m_linea,wind_dir_100m_linea,wind_gusts_linea,soil_temperature_linea
0,3014_278962,16:42:00,16:42:00,S06404_1,1,0.00000,0.000000,0.000000,0.00000,18586,...,0.0,93,0.5,53,8.3,14.6,72,81,15.1,11.1
1,3014_278962,16:48:00,16:49:00,S06402_1,2,8.01853,0.100000,80.185300,8.01853,18586,...,0.0,93.0,0.5,53.0,8.3,14.6,72.0,81.0,15.1,11.1
2,3014_278962,16:57:00,16:58:00,S06400_1,3,14.83468,0.133333,51.121125,6.81615,18586,...,0.0,94.5,0.3,52.0,6.9,13.5,69.5,77.0,15.45,10.8
3,3014_278962,17:08:00,17:09:00,S06501_1,4,20.90887,0.166667,36.445140,6.07419,18586,...,0.0,96.5,0.05,27.0,6.3,13.85,71.0,77.0,15.45,10.35
4,3014_278962,17:13:00,17:13:00,S06500_1,5,23.53195,0.066667,39.346200,2.62308,18586,...,0.0,97.0,0.0,3.0,7.1,15.3,75.0,81.0,15.1,10.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291994,4118_296638,17:42:00,17:42:00,S06404_1,1,0.00000,0.000000,0.000000,0.00000,18594,...,0.0,73,0.4,51,16.9,30.5,259,256,32.4,17.0
291995,4118_296638,17:48:00,17:49:00,S06402_1,2,8.01853,0.100000,80.185300,8.01853,18594,...,0.0,73.0,0.4,51.0,16.9,30.5,259.0,256.0,32.4,17.0
291996,4118_296638,17:57:00,17:58:00,S06400_1,3,14.83468,0.133333,51.121125,6.81615,18594,...,0.0,74.5,0.2,26.5,14.65,28.35,259.5,256.5,34.2,17.15
291997,4118_296638,18:07:00,18:08:00,S06501_1,4,20.90887,0.150000,40.494600,6.07419,18594,...,0.0,73.5,0.0,2.0,13.95,28.45,259.0,256.5,33.85,17.7


In [17]:
# Drop Duplicates
row_to_delete =[]  
# Change orario realtime arrivo stazione 1 (non c'è to = arrivo_schedulato)
for index, row  in tqdm(main_dataset.iterrows()):
    # X nessun ritardo, N significa che non riparte perché era l'ultima destinazione oppure quella di partenza
    if((row['ritardo_arrivo'] == 'X') |( row['ritardo_arrivo'] == 'N')):
        main_dataset.loc[index,"ritardo_arrivo"] = '0'
    if((row['ritardo_partenza'] == 'X' )| (row['ritardo_partenza'] == 'N')):
        main_dataset.loc[index,"ritardo_partenza"] = '0'
    if((row['ritardo_arrivo'] == 'S') | (row['ritardo_arrivo'] == 'n.d.')):
        row_to_delete.append(index)
    if((row['ritardo_partenza'] == 'S') | (row['ritardo_partenza'] == 'n.d.')):
        row_to_delete.append(index)

row_to_delete = list(dict.fromkeys(row_to_delete))
main_dataset = main_dataset[~main_dataset.index.isin(row_to_delete)]
main_dataset.reset_index(drop=True)

291999it [00:57, 5086.08it/s] 


,trip_id,arrival_time,departure_time,codice_stazione,stop_sequence,shape_dist_traveled,time_diff,speed,dist_diff,codice,...,snow_depth_linea,humidity_linea,precipitation_linea,wmo_code_linea,wind_speed_10m_linea,wind_speed_100m_linea,wind_dir_10m_linea,wind_dir_100m_linea,wind_gusts_linea,soil_temperature_linea
0,3014_278962,16:42:00,16:42:00,S06404_1,1,0.00000,0.000000,0.000000,0.00000,18586,...,0.0,93,0.5,53,8.3,14.6,72,81,15.1,11.1
1,3014_278962,16:48:00,16:49:00,S06402_1,2,8.01853,0.100000,80.185300,8.01853,18586,...,0.0,93.0,0.5,53.0,8.3,14.6,72.0,81.0,15.1,11.1
2,3014_278962,16:57:00,16:58:00,S06400_1,3,14.83468,0.133333,51.121125,6.81615,18586,...,0.0,94.5,0.3,52.0,6.9,13.5,69.5,77.0,15.45,10.8
3,3014_278962,17:08:00,17:09:00,S06501_1,4,20.90887,0.166667,36.445140,6.07419,18586,...,0.0,96.5,0.05,27.0,6.3,13.85,71.0,77.0,15.45,10.35
4,3014_278962,17:13:00,17:13:00,S06500_1,5,23.53195,0.066667,39.346200,2.62308,18586,...,0.0,97.0,0.0,3.0,7.1,15.3,75.0,81.0,15.1,10.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291768,4118_296638,17:42:00,17:42:00,S06404_1,1,0.00000,0.000000,0.000000,0.00000,18594,...,0.0,73,0.4,51,16.9,30.5,259,256,32.4,17.0
291769,4118_296638,17:48:00,17:49:00,S06402_1,2,8.01853,0.100000,80.185300,8.01853,18594,...,0.0,73.0,0.4,51.0,16.9,30.5,259.0,256.0,32.4,17.0
291770,4118_296638,17:57:00,17:58:00,S06400_1,3,14.83468,0.133333,51.121125,6.81615,18594,...,0.0,74.5,0.2,26.5,14.65,28.35,259.5,256.5,34.2,17.15
291771,4118_296638,18:07:00,18:08:00,S06501_1,4,20.90887,0.150000,40.494600,6.07419,18594,...,0.0,73.5,0.0,2.0,13.95,28.45,259.0,256.5,33.85,17.7


In [18]:
main_dataset["ritardo_partenza"] = main_dataset["ritardo_partenza"].astype(int)
main_dataset["ritardo_arrivo"] = main_dataset["ritardo_arrivo"].astype(int)
max_index_for_display = main_dataset.index.max()

for index, row  in tqdm(main_dataset.iterrows()):
    main_dataset.loc[index,'arrival_time'] = row['arrival_time'].replace(':','')
    main_dataset.loc[index,'departure_time'] = row['departure_time'].replace(':','')
    main_dataset.loc[index,'stop_code'] = row['stop_code'].replace('S','')
    main_dataset.loc[index,'time_diff'] = round(row['time_diff'], 2)
    main_dataset.loc[index,'dist_diff'] = round(row['dist_diff'], 2)
    main_dataset.loc[index,'speed'] = round(row['speed'], 2)
    
    if row['ritardo_arrivo'] < 0:
        main_dataset.loc[index,'ritardo_arrivo'] = 0
    if row['ritardo_partenza'] < 0:
        main_dataset.loc[index,'ritardo_partenza'] = 0


291773it [17:38, 275.75it/s]


In [19]:
main_dataset

,trip_id,arrival_time,departure_time,codice_stazione,stop_sequence,shape_dist_traveled,time_diff,speed,dist_diff,codice,...,snow_depth_linea,humidity_linea,precipitation_linea,wmo_code_linea,wind_speed_10m_linea,wind_speed_100m_linea,wind_dir_10m_linea,wind_dir_100m_linea,wind_gusts_linea,soil_temperature_linea
0,3014_278962,164200,164200,S06404_1,1,0.00000,0.00,0.00,0.00,18586,...,0.0,93,0.5,53,8.3,14.6,72,81,15.1,11.1
1,3014_278962,164800,164900,S06402_1,2,8.01853,0.10,80.19,8.02,18586,...,0.0,93.0,0.5,53.0,8.3,14.6,72.0,81.0,15.1,11.1
2,3014_278962,165700,165800,S06400_1,3,14.83468,0.13,51.12,6.82,18586,...,0.0,94.5,0.3,52.0,6.9,13.5,69.5,77.0,15.45,10.8
3,3014_278962,170800,170900,S06501_1,4,20.90887,0.17,36.45,6.07,18586,...,0.0,96.5,0.05,27.0,6.3,13.85,71.0,77.0,15.45,10.35
4,3014_278962,171300,171300,S06500_1,5,23.53195,0.07,39.35,2.62,18586,...,0.0,97.0,0.0,3.0,7.1,15.3,75.0,81.0,15.1,10.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291994,4118_296638,174200,174200,S06404_1,1,0.00000,0.00,0.00,0.00,18594,...,0.0,73,0.4,51,16.9,30.5,259,256,32.4,17.0
291995,4118_296638,174800,174900,S06402_1,2,8.01853,0.10,80.19,8.02,18594,...,0.0,73.0,0.4,51.0,16.9,30.5,259.0,256.0,32.4,17.0
291996,4118_296638,175700,175800,S06400_1,3,14.83468,0.13,51.12,6.82,18594,...,0.0,74.5,0.2,26.5,14.65,28.35,259.5,256.5,34.2,17.15
291997,4118_296638,180700,180800,S06501_1,4,20.90887,0.15,40.49,6.07,18594,...,0.0,73.5,0.0,2.0,13.95,28.45,259.0,256.5,33.85,17.7


In [20]:
main_dataset.to_csv("../processed_files/main_dataset.csv")